In [ ]:
import pyspark
from pyspark.sql.functions import current_date
from cassandra.cluster import Cluster
from datetime import datetime
from pyspark.sql.functions import lit, monotonically_increasing_id

In [ ]:
spark = pyspark.sql.SparkSession.builder \
    .appName('sample') \
    .config("spark.jars", "/home/docker_worker/libs/spark-cassandra-connector-assembly_2.12-3.0-9.jar") \
    .config('spark.cassandra.connection.host', '172.24.160.3') \
    .config('spark.cassandra.connection.port', '9042') \
    .config('spark.cassandra.connection.timeout_ms', '5000') \
    .config('spark.cassandra.read.timeout_ms', '200000') \
    .config('spark.cassandra.connection.localDC', 'DC1') \
    .config('spark.cassandra.output.consistency.level','ONE') \
    .config('local-datacenter = datacenter1') \
    .getOrCreate()

In [ ]:
file = '/home/docker_worker/work/twitter_arq.txt'

In [ ]:
data_hora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
df = spark.read.option("delimiter", ",").option("header","true").csv(file).drop('_c0').withColumnRenamed('text', 'obs').withColumn("numseq", monotonically_increasing_id())

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.write.format("org.apache.spark.sql.cassandra").mode("overwrite").option("confirm.truncate","true").options(keyspace="twitters",table="twittarquivo").save()

In [ ]:
spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="twittarquivo", keyspace="twitters") \
    .load().show()